# Download MARBERT checkpoint

In [ ]:
!wget https://huggingface.co/UBC-NLP/MARBERT/resolve/main/MARBERT_pytorch_verison.tar.gz

--2024-06-16 22:27:03--  https://huggingface.co/UBC-NLP/MARBERT/resolve/main/MARBERT_pytorch_verison.tar.gz
Resolving huggingface.co (huggingface.co)... 3.163.189.37, 3.163.189.114, 3.163.189.74, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.37|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/UBC-NLP/MARBERT/85bfec76f38cba4bc2e6cd02a959016de37ba93de4c850a7d175811dce4e8adc?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27MARBERT_pytorch_verison.tar.gz%3B+filename%3D%22MARBERT_pytorch_verison.tar.gz%22%3B&response-content-type=application%2Fgzip&Expires=1718836023&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxODgzNjAyM319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9VQkMtTkxQL01BUkJFUlQvODViZmVjNzZmMzhjYmE0YmMyZTZjZDAyYTk1OTAxNmRlMzdiYTkzZGU0Yzg1MGE3ZDE3NTgxMWRjZTRlOGFkYz9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSomcmVzcG9uc2UtY29udGVudC10eXBlPSoi

In [ ]:
!tar -xvf MARBERT_pytorch_verison.tar.gz

MARBERT_pytorch_verison/
MARBERT_pytorch_verison/pytorch_model.bin
MARBERT_pytorch_verison/config.json
MARBERT_pytorch_verison/vocab.txt


In [ ]:
!wget https://raw.githubusercontent.com/UBC-NLP/marbert/main/examples/UBC_AJGT_final_shuffled_train.tsv
!wget https://raw.githubusercontent.com/UBC-NLP/marbert/main/examples/UBC_AJGT_final_shuffled_test.tsv

--2024-06-16 22:27:15--  https://raw.githubusercontent.com/UBC-NLP/marbert/main/examples/UBC_AJGT_final_shuffled_train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149495 (146K) [text/plain]
Saving to: ‘UBC_AJGT_final_shuffled_train.tsv’

UBC_AJGT_final_shuf 100%[===================>] 145.99K  --.-KB/s    in 0.02s   

2024-06-16 22:27:15 (8.02 MB/s) - ‘UBC_AJGT_final_shuffled_train.tsv’ saved [149495/149495]

--2024-06-16 22:27:16--  https://raw.githubusercontent.com/UBC-NLP/marbert/main/examples/UBC_AJGT_final_shuffled_test.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
H

In [ ]:
!mkdir -p AJGT
!mv UBC_AJGT_final_shuffled_train.tsv ./AJGT/UBC_AJGT_final_shuffled_train.tsv
!mv UBC_AJGT_final_shuffled_test.tsv ./AJGT/UBC_AJGT_final_shuffled_test.tsv

In [ ]:
!pip install GPUtil pytorch_pretrained_bert transformers

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cach

# Fine-tuning code

In [ ]:
# (1)load libraries
import json, sys, regex
import torch
import GPUtil
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig, BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import os
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix
##----------------------------------------------------
from transformers import *
from transformers import XLMRobertaConfig
from transformers import XLMRobertaModel
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, XLMRobertaModel
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel


/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("your device ", device)

your device  cuda


In [ ]:

def create_label2ind_file(file, label_col):
	labels_json={}
	#load train_dev_test file
	df = pd.read_csv(file, sep="\t")
	df.head(5)
	#get labels and sort it A-Z
	labels = df[label_col].unique()
	labels.sort()
	#convert labels to indexes
	for idx in range(0, len(labels)):
		labels_json[labels[idx]]=idx
	#save labels with indexes to file
	with open(label2idx_file, 'w') as json_file:
		json.dump(labels_json, json_file)


In [ ]:

def data_prepare_BERT(file_path, lab2ind, tokenizer, content_col, label_col, MAX_LEN):
	# Use pandas to load dataset
	df = pd.read_csv(file_path, delimiter='\t', header=0)
	df = df[df[content_col].notnull()]
	df = df[df[label_col].notnull()]
	print("Data size ", df.shape)
	# Create sentence and label lists
	sentences = df[content_col].values
	sentences = ["[CLS] " + sentence+ " [SEP]" for sentence in sentences]
	print ("The first sentence:")
	print (sentences[0])
	# Create sentence and label lists
	labels = df[label_col].values
	#print (labels)
	labels = [lab2ind[i] for i in labels]
	# Import the BERT tokenizer, used to convert our text into tokens that correspond to BERT's vocabulary.
	tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
	print ("Tokenize the first sentence:")
	print (tokenized_texts[0])
	#print("Label is ", labels[0])
	# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
	input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
	print ("Index numbers of the first sentence:")
	print (input_ids[0])
	# Pad our input seqeunce to the fixed length (i.e., max_len) with index of [PAD] token
	# ~ input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
	pad_ind = tokenizer.convert_tokens_to_ids(['[PAD]'])[0]
	input_ids = pad_sequences(input_ids, maxlen=MAX_LEN+2, dtype="long", truncating="post", padding="post", value=pad_ind)
	print ("Index numbers of the first sentence after padding:\n",input_ids[0])
	# Create attention masks
	attention_masks = []
	# Create a mask of 1s for each token followed by 0s for padding
	for seq in input_ids:
		seq_mask = [float(i > 0) for i in seq]
		attention_masks.append(seq_mask)
	# Convert all of our data into torch tensors, the required datatype for our model
	inputs = torch.tensor(input_ids)
	labels = torch.tensor(labels)
	masks = torch.tensor(attention_masks)
	return inputs, labels, masks


In [ ]:
# Function to calculate the accuracy of our predictions vs labels
# def flat_accuracy(preds, labels):
#	  pred_flat = np.argmax(preds, axis=1).flatten()
#	  labels_flat = labels.flatten()
#	  return np.sum(pred_flat == labels_flat) / len(labels_flat)
def flat_pred(preds, labels):
	pred_flat = np.argmax(preds, axis=1).flatten()
	labels_flat = labels.flatten()
	return pred_flat.tolist(), labels_flat.tolist()

In [ ]:

def train(model, iterator, optimizer, scheduler, criterion):

	model.train()
	epoch_loss = 0
	for i, batch in enumerate(iterator):
		# Add batch to GPU
		batch = tuple(t.to(device) for t in batch)
		# Unpack the inputs from our dataloader
		input_ids, input_mask, labels = batch
		outputs = model(input_ids, input_mask, labels=labels)
		loss, logits = outputs[:2]
		# delete used variables to free GPU memory
		del batch, input_ids, input_mask, labels
		optimizer.zero_grad()
		if torch.cuda.device_count() == 1:
			loss.backward()
			epoch_loss += loss.cpu().item()
		else:
			loss.sum().backward()
			epoch_loss += loss.sum().cpu().item()
		optimizer.step()
		torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)  # Gradient clipping is not in AdamW anymore
		# optimizer.step()
		scheduler.step()
	# free GPU memory
	if device == 'cuda':
		torch.cuda.empty_cache()
	return epoch_loss / len(iterator)

In [ ]:

def evaluate(model, iterator, criterion):
	model.eval()
	epoch_loss = 0
	all_pred=[]
	all_label = []
	with torch.no_grad():
		for i, batch in enumerate(iterator):
			# Add batch to GPU
			batch = tuple(t.to(device) for t in batch)
			# Unpack the inputs from our dataloader
			input_ids, input_mask, labels = batch
			outputs = model(input_ids, input_mask, labels=labels)
			loss, logits = outputs[:2]
			# delete used variables to free GPU memory
			del batch, input_ids, input_mask
			if torch.cuda.device_count() == 1:
				epoch_loss += loss.cpu().item()
			else:
				epoch_loss += loss.sum().cpu().item()
			# identify the predicted class for each example in the batch
			probabilities, predicted = torch.max(logits.cpu().data, 1)
			# put all the true labels and predictions to two lists
			all_pred.extend(predicted)
			all_label.extend(labels.cpu())
	accuracy = accuracy_score(all_label, all_pred)
	f1score = f1_score(all_label, all_pred, average='macro')
	recall = recall_score(all_label, all_pred, average='macro')
	precision = precision_score(all_label, all_pred, average='macro')
	report = classification_report(all_label, all_pred)
	return (epoch_loss / len(iterator)), accuracy, f1score, recall, precision



In [ ]:

def fine_tuning(config):
	#---------------------------------------
	print ("[INFO] step (1) load train_test config file")
	# config_file = open(config_file, 'r', encoding="utf8")
	# config = json.load(config_file)
	task_name = config["task_name"]
	content_col = config["content_col"]
	label_col = config["label_col"]
	train_file = config["data_dir"]+config["train_file"]
	dev_file = config["data_dir"]+config["dev_file"]
	sortby = config["sortby"]
	max_seq_length= int(config["max_seq_length"])
	batch_size = int(config["batch_size"])
	lr_var = float(config["lr"])
	model_path = config['pretrained_model_path']
	num_epochs = config['epochs'] # Number of training epochs (authors recommend between 2 and 4)
	global label2idx_file
	label2idx_file = config["data_dir"]+config["task_name"]+"_labels-dict.json"
	#-------------------------------------------------------
	print ("[INFO] step (2) convert labels2index")
	create_label2ind_file(train_file, label_col)
	print (label2idx_file)
	#---------------------------------------------------------
	print ("[INFO] step (3) check checkpoit directory and report file")
	ckpt_dir = config["data_dir"]+task_name+"_bert_ckpt/"
	report = ckpt_dir+task_name+"_report.tsv"
	sorted_report = ckpt_dir+task_name+"_report_sorted.tsv"
	if not os.path.exists(ckpt_dir):
		os.mkdir(ckpt_dir)
	#-------------------------------------------------------
	print ("[INFO] step (4) load label to number dictionary")
	lab2ind = json.load(open(label2idx_file))
	print ("[INFO] train_file", train_file)
	print ("[INFO] dev_file", dev_file)
	print ("[INFO] num_epochs", num_epochs)
	print ("[INFO] model_path", model_path)
	print ("max_seq_length", max_seq_length, "batch_size", batch_size)
	#-------------------------------------------------------
	print ("[INFO] step (5) Use defined funtion to extract tokanize data")
	# tokenizer from pre-trained BERT model
	print ("loading BERT setting")
	tokenizer = BertTokenizer.from_pretrained(model_path)
	train_inputs, train_labels, train_masks = data_prepare_BERT(train_file, lab2ind, tokenizer,content_col, label_col, max_seq_length)
	validation_inputs, validation_labels, validation_masks = data_prepare_BERT(dev_file, lab2ind, tokenizer, content_col, label_col,max_seq_length)
	# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top.
	model = BertForSequenceClassification.from_pretrained(model_path, num_labels=len(lab2ind))
	#--------------------------------------
	print ("[INFO] step (6) Create an iterator of data with torch DataLoader.")
#		  This helps save on memory during training because, unlike a for loop,\
#		  with an iterator the entire dataset does not need to be loaded into memory")
	train_data = TensorDataset(train_inputs, train_masks, train_labels)
	train_dataloader = DataLoader(train_data, batch_size=batch_size)
	#---------------------------
	validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
	validation_dataloader = DataLoader(validation_data, batch_size=batch_size)
	#------------------------------------------
	print ("[INFO] step (7) run with parallel GPUs")
	if torch.cuda.is_available():
		if torch.cuda.device_count() == 1:
			print("Run", "with one GPU")
			model = model.to(device)
		else:
			n_gpu = torch.cuda.device_count()
			print("Run", "with", n_gpu, "GPUs with max 4 GPUs")
			device_ids = GPUtil.getAvailable(limit = 4)
			torch.backends.cudnn.benchmark = True
			model = model.to(device)
			model = nn.DataParallel(model, device_ids=device_ids)
	else:
		print("Run", "with CPU")
		model = model
	#---------------------------------------------------
	print ("[INFO] step (8) set Parameters, schedules, and loss function")
	global max_grad_norm
	max_grad_norm = 1.0
	warmup_proportion = 0.1
	num_training_steps	= len(train_dataloader) * num_epochs
	num_warmup_steps = num_training_steps * warmup_proportion
	### In Transformers, optimizer and schedules are instantiated like this:
	# Note: AdamW is a class from the huggingface library
	# the 'W' stands for 'Weight Decay"
	optimizer = AdamW(model.parameters(), lr=lr_var, correct_bias=False)
	# schedules
	scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)  # PyTorch scheduler
	# We use nn.CrossEntropyLoss() as our loss function.
	criterion = nn.CrossEntropyLoss()
	#---------------------------------------------------
	print ("[INFO] step (9) start fine_tuning")
	for epoch in trange(num_epochs, desc="Epoch"):
		train_loss = train(model, train_dataloader, optimizer, scheduler, criterion)
		val_loss, val_acc, val_f1, val_recall, val_precision = evaluate(model, validation_dataloader, criterion)
# 		print (train_loss, val_acc)
		# Create checkpoint at end of each epoch
		if not os.path.exists(ckpt_dir + 'model_' + str(int(epoch + 1)) + '/'): os.mkdir(ckpt_dir + 'model_' + str(int(epoch + 1)) + '/')
		model.save_pretrained(ckpt_dir+ 'model_' + str(int(epoch + 1)) + '/')
		epoch_eval_results = {"epoch_num":int(epoch + 1),"train_loss":train_loss,
					  "val_acc":val_acc, "val_recall":val_recall, "val_precision":val_precision, "val_f1":val_f1,"lr":lr_var }
		with open(report,"a") as fOut:
			fOut.write(json.dumps(epoch_eval_results)+"\n")
			fOut.flush()
		#------------------------------------
		report_df = pd.read_json(report, orient='records', lines=True)
		report_df.sort_values(by=[sortby],ascending=False, inplace=True)
		report_df.to_csv(sorted_report,sep="\t",index=False)
	return report_df

# Run fine-tuning for 5 epochs

In [ ]:

config={"task_name": "AJGT_MARBERT", #output directory name
             "data_dir": "./AJGT/", #data directory
             "train_file": "UBC_AJGT_final_shuffled_train.tsv", #train file path
             "dev_file": "UBC_AJGT_final_shuffled_test.tsv", #dev file path or test file path
             "pretrained_model_path": 'MARBERT_pytorch_verison', #MARBERT checkpoint path
             "epochs": 36, #number of epochs
             "content_col": "content", #text column
             "label_col": "label", #label column
             "lr": 2e-06, #learning rate
              "max_seq_length": 128, #max sequance length
              "batch_size": 32, #batch shize
              "sortby":"val_acc"} #sort results based on val_acc or val_f1


In [ ]:
report_df = fine_tuning(config)

loading file vocab.txt
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading file tokenizer.json
loading configuration file MARBERT_pytorch_verison/config.json


[INFO] step (1) load train_test config file
[INFO] step (2) convert labels2index
./AJGT/AJGT_MARBERT_labels-dict.json
[INFO] step (3) check checkpoit directory and report file
[INFO] step (4) load label to number dictionary
[INFO] train_file ./AJGT/UBC_AJGT_final_shuffled_train.tsv
[INFO] dev_file ./AJGT/UBC_AJGT_final_shuffled_test.tsv
[INFO] num_epochs 36
[INFO] model_path MARBERT_pytorch_verison
max_seq_length 128 batch_size 32
[INFO] step (5) Use defined funtion to extract tokanize data
loading BERT setting
Data size  (1440, 2)
The first sentence:
[CLS]  اللهم  ارزقنا  فعل  الخيرات  وترك  المنكرات [SEP]
Tokenize the first sentence:
['[CLS]', 'اللهم', 'ارزقنا', 'فعل', 'الخيرات', 'وترك', 'المنكرات', '[SEP]']
Index numbers of the first sentence:
[2, 1983, 4246, 4176, 11736, 7872, 28445, 3]
Index numbers of the first sentence after padding:
 [    2  1983  4246  4176 11736  7872 28445     3     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0


loading configuration file MARBERT_pytorch_verison/config.json
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.41.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}

loading weights file MARBERT_pytorch_verison/pytorch_model.bin


Tokenize the first sentence:
['[CLS]', 'فما', 'اجمل', 'هذا', 'الدين', '[SEP]']
Index numbers of the first sentence:
[2, 3880, 2605, 2158, 2926, 3]
Index numbers of the first sentence after padding:
 [   2 3880 2605 2158 2926    3    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]


Some weights of the model checkpoint at MARBERT_pytorch_verison were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpo

[INFO] step (6) Create an iterator of data with torch DataLoader.
[INFO] step (7) run with parallel GPUs
Run with one GPU


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


[INFO] step (8) set Parameters, schedules, and loss function
[INFO] step (9) start fine_tuning


Epoch:   0%|          | 0/36 [00:00<?, ?it/s]Configuration saved in ./AJGT/AJGT_MARBERT_bert_ckpt/model_1/config.json
Model weights saved in ./AJGT/AJGT_MARBERT_bert_ckpt/model_1/model.safetensors
Epoch:   3%|▎         | 1/36 [00:37<21:40, 37.16s/it]Configuration saved in ./AJGT/AJGT_MARBERT_bert_ckpt/model_2/config.json
Model weights saved in ./AJGT/AJGT_MARBERT_bert_ckpt/model_2/model.safetensors
Epoch:   6%|▌         | 2/36 [01:12<20:26, 36.08s/it]Configuration saved in ./AJGT/AJGT_MARBERT_bert_ckpt/model_3/config.json
Model weights saved in ./AJGT/AJGT_MARBERT_bert_ckpt/model_3/model.safetensors
Epoch:   8%|▊         | 3/36 [01:47<19:32, 35.54s/it]Configuration saved in ./AJGT/AJGT_MARBERT_bert_ckpt/model_4/config.json
Model weights saved in ./AJGT/AJGT_MARBERT_bert_ckpt/model_4/model.safetensors
Epoch:  11%|█         | 4/36 [02:24<19:14, 36.07s/it]Configuration saved in ./AJGT/AJGT_MARBERT_bert_ckpt/model_5/config.json
Model weights saved in ./AJGT/AJGT_MARBERT_bert_ckpt/model_5/m

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
report_df.head(36)

,epoch_num,train_loss,val_acc,val_recall,val_precision,val_f1,lr
18,19,0.007903,0.961111,0.960965,0.960965,0.960965,0.000002
25,26,0.004366,0.961111,0.960965,0.960965,0.960965,0.000002
15,16,0.007901,0.961111,0.960965,0.960965,0.960965,0.000002
13,14,0.009370,0.961111,0.960965,0.960965,0.960965,0.000002
12,13,0.011037,0.958333,0.958348,0.958050,0.958191,0.000002
11,12,0.011980,0.958333,0.958007,0.958333,0.958163,0.000002
10,11,0.013332,0.958333,0.957666,0.958732,0.958131,0.000002
30,31,0.002933,0.955556,0.955389,0.955389,0.955389,0.000002
9,10,0.015830,0.955556,0.954707,0.956177,0.955323,0.000002
20,21,0.008320,0.955556,0.955730,0.955166,0.955418,0.000002


In [ ]:
import os

ckpt_dir = "/content/AJGT/AJGT_MARBERT_bert_ckpt"
contents = os.listdir(ckpt_dir)
print(contents)


['model_18', 'model_29', 'model_31', 'model_23', 'model_5', 'model_35', 'model_10', 'model_36', 'model_14', 'model_30', 'model_12', 'model_27', 'model_2', 'model_20', 'model_1', 'AJGT_MARBERT_report.tsv', 'model_22', 'model_15', 'model_21', 'model_7', 'model_3', 'model_13', 'model_34', 'model_25', 'model_24', 'model_11', 'model_8', 'model_17', 'model_6', 'model_33', 'model_19', 'model_16', 'model_9', 'AJGT_MARBERT_report_sorted.tsv', 'model_28', 'model_26', 'model_32', 'model_4']


In [ ]:
import os

ckpt_dir = "/content/AJGT/AJGT_MARBERT_bert_ckpt"
for subdir in os.listdir(ckpt_dir):
    subdir_path = os.path.join(ckpt_dir, subdir)
    if os.path.isdir(subdir_path):
        print(f"Contents of {subdir}:")
        print(os.listdir(subdir_path))


Contents of model_18:
['model.safetensors', 'config.json']
Contents of model_29:
['model.safetensors', 'config.json']
Contents of model_31:
['model.safetensors', 'config.json']
Contents of model_23:
['model.safetensors', 'config.json']
Contents of model_5:
['model.safetensors', 'config.json']
Contents of model_35:
['model.safetensors', 'config.json']
Contents of model_10:
['model.safetensors', 'config.json']
Contents of model_36:
['model.safetensors', 'config.json']
Contents of model_14:
['model.safetensors', 'config.json']
Contents of model_30:
['model.safetensors', 'config.json']
Contents of model_12:
['model.safetensors', 'config.json']
Contents of model_27:
['model.safetensors', 'config.json']
Contents of model_2:
['model.safetensors', 'config.json']
Contents of model_20:
['model.safetensors', 'config.json']
Contents of model_1:
['model.safetensors', 'config.json']
Contents of model_22:
['model.safetensors', 'config.json']
Contents of model_15:
['model.safetensors', 'config.json']


In [ ]:
import torch
from transformers import BertForSequenceClassification

ckpt_dir = "/content/AJGT/AJGT_MARBERT_bert_ckpt"

# Iterate over the subdirectories and load the model
for subdir in os.listdir(ckpt_dir):
    subdir_path = os.path.join(ckpt_dir, subdir)
    if os.path.isdir(subdir_path):
        # Load the model
        model_path = os.path.join(subdir_path, "model.safetensors")
        model = BertForSequenceClassification.from_pretrained(subdir_path)

        # Check if the model loaded successfully
        if model:
            print(f"Model loaded successfully from {model_path}")
        else:
            print(f"Failed to load model from {model_path}")


loading configuration file /content/AJGT/AJGT_MARBERT_bert_ckpt/model_18/config.json
Model config BertConfig {
  "_name_or_path": "MARBERT_pytorch_verison",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.41.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}



Model loaded successfully from /content/AJGT/AJGT_MARBERT_bert_ckpt/model_18/model.safetensors


All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the model checkpoint at /content/AJGT/AJGT_MARBERT_bert_ckpt/model_29.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.
loading configuration file /content/AJGT/AJGT_MARBERT_bert_ckpt/model_31/config.json
Model config BertConfig {
  "_name_or_path": "MARBERT_pytorch_verison",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad

Model loaded successfully from /content/AJGT/AJGT_MARBERT_bert_ckpt/model_29/model.safetensors


All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the model checkpoint at /content/AJGT/AJGT_MARBERT_bert_ckpt/model_31.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.
loading configuration file /content/AJGT/AJGT_MARBERT_bert_ckpt/model_23/config.json
Model config BertConfig {
  "_name_or_path": "MARBERT_pytorch_verison",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad

...

Model loaded successfully from /content/AJGT/AJGT_MARBERT_bert_ckpt/model_26/model.safetensors


All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the model checkpoint at /content/AJGT/AJGT_MARBERT_bert_ckpt/model_32.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.
loading configuration file /content/AJGT/AJGT_MARBERT_bert_ckpt/model_4/config.json
Model config BertConfig {
  "_name_or_path": "MARBERT_pytorch_verison",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_

Model loaded successfully from /content/AJGT/AJGT_MARBERT_bert_ckpt/model_32/model.safetensors


All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the model checkpoint at /content/AJGT/AJGT_MARBERT_bert_ckpt/model_4.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.


Model loaded successfully from /content/AJGT/AJGT_MARBERT_bert_ckpt/model_4/model.safetensors


In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader
import torch

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('MARBERT_pytorch_verison')

# Load model
model_path = "/content/AJGT/AJGT_MARBERT_bert_ckpt/model_36"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained(model_path)
model.to(device)

# Assuming you have a new dataset in an Excel file named "new_data.xlsx"
new_data_file = "/content/drive/My Drive/my grad/new_data.xlsx"

# Check the column names in the DataFrame
new_data = pd.read_excel(new_data_file)
print(new_data.columns)

# Tokenize and prepare the new dataset
encoded_data = tokenizer(new_data['content'].tolist(), truncation=True, padding=True, return_tensors='pt')
input_ids = encoded_data['input_ids']
attention_masks = encoded_data['attention_mask']
dataset = TensorDataset(input_ids, attention_masks)

# Define dataloader
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size)

# Perform prediction
predictions = []
model.eval()
for batch in dataloader:
    batch = tuple(t.to(device) for t in batch)
    with torch.no_grad():
        outputs = model(*batch)
    logits = outputs[0]
    predictions.extend(torch.argmax(logits, dim=1).tolist())

# Now, `predictions` contains the predicted labels for your new dataset


loading file vocab.txt
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading file tokenizer.json
loading configuration file MARBERT_pytorch_verison/config.json
loading configuration file /content/AJGT/AJGT_MARBERT_bert_ckpt/model_36/config.json
Model config BertConfig {
  "_name_or_path": "MARBERT_pytorch_verison",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "posit

Index(['content'], dtype='object')


In [ ]:
# Perform prediction
predictions = []
model.eval()
for batch in dataloader:
    batch = tuple(t.to(device) for t in batch)
    with torch.no_grad():
        outputs = model(*batch)
    logits = outputs[0]
    predictions.extend(torch.argmax(logits, dim=1).tolist())

# Now, `predictions` contains the predicted labels for your new dataset

# Print the predictions
print("Predictions:")
for idx, label in enumerate(predictions):
    print(f"Sample {idx + 1}: Predicted Label = {label}")


Predictions:
Sample 1: Predicted Label = 0
Sample 2: Predicted Label = 0
Sample 3: Predicted Label = 1
Sample 4: Predicted Label = 0
Sample 5: Predicted Label = 0
Sample 6: Predicted Label = 1
Sample 7: Predicted Label = 0
Sample 8: Predicted Label = 0
Sample 9: Predicted Label = 1
Sample 10: Predicted Label = 1
Sample 11: Predicted Label = 0
Sample 12: Predicted Label = 0
Sample 13: Predicted Label = 0
Sample 14: Predicted Label = 0
Sample 15: Predicted Label = 0
Sample 16: Predicted Label = 1
Sample 17: Predicted Label = 0
Sample 18: Predicted Label = 0
Sample 19: Predicted Label = 0
Sample 20: Predicted Label = 1
Sample 21: Predicted Label = 1
Sample 22: Predicted Label = 1
Sample 23: Predicted Label = 1
Sample 24: Predicted Label = 1


In [ ]:
# Define a mapping of predicted indices to negative or positive labels
label_mapping = {
    0: "Negative",
    1: "Positive"
}

# Perform prediction
predictions = []
model.eval()
for batch in dataloader:
    batch = tuple(t.to(device) for t in batch)
    with torch.no_grad():
        outputs = model(*batch)
    logits = outputs[0]
    predictions.extend(torch.argmax(logits, dim=1).tolist())

# Now, `predictions` contains the predicted labels for your new dataset

# Convert predicted indices to negative or positive labels using the mapping
converted_predictions = [label_mapping[idx] for idx in predictions]

# Print the converted predictions
print("Predictions:")
for idx, label in enumerate(converted_predictions):
    print(f"Sample {idx + 1}: Predicted Label = {label}")


Predictions:
Sample 1: Predicted Label = Negative
Sample 2: Predicted Label = Negative
Sample 3: Predicted Label = Positive
Sample 4: Predicted Label = Negative
Sample 5: Predicted Label = Negative
Sample 6: Predicted Label = Positive
Sample 7: Predicted Label = Negative
Sample 8: Predicted Label = Negative
Sample 9: Predicted Label = Positive
Sample 10: Predicted Label = Positive
Sample 11: Predicted Label = Negative
Sample 12: Predicted Label = Negative
Sample 13: Predicted Label = Negative
Sample 14: Predicted Label = Negative
Sample 15: Predicted Label = Negative
Sample 16: Predicted Label = Positive
Sample 17: Predicted Label = Negative
Sample 18: Predicted Label = Negative
Sample 19: Predicted Label = Negative
Sample 20: Predicted Label = Positive
Sample 21: Predicted Label = Positive
Sample 22: Predicted Label = Positive
Sample 23: Predicted Label = Positive
Sample 24: Predicted Label = Positive
